# Deep Deterministic Policy Gradients (DDPG)
---
Notebook, addapted from 
https://raw.githubusercontent.com/udacity/deep-reinforcement-learning/master/ddpg-bipedal/DDPG.ipynb
training DDPG with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Import the Necessary Packages

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.28 which is incompatible.


In [2]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from DDPG_Multi_agent_kthStep import Agent

### 2. Instantiate the Environment and Agent

In [3]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
## env = UnityEnvironment(file_name='C:\EigeneLokaleDaten\DeepRL\Value-based-methods\p2_continuous-control\Reacher_Windows_x86_64_multiple_agents\Reacher.exe')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [6]:
#env = gym.make('BipedalWalker-v2')
#env.seed(10)
#agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], random_seed=10)

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the 2nd agent looks like:', states[1])

print(state_size)
print(len(states))

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]
The state for the 2nd agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -

In [8]:
NN_architecture = None # default 256,256,256,128
#NN_architecture = [64,64,32,16]  # actor_fc, critic_fc1,critic_fc2,critic_fc3
#NN_architecture = [16,32,16,8]  # actor_fc, critic_fc1,critic_fc2,critic_fc3

agent = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, random_seed=10, NN_architecture=NN_architecture)

Hyperparams 1000000 128 0.99 0.001 0.0001 0.001 0 new update 30 20
Hyperparams Noise 0.0 0.15 0.2


In [9]:
print(agent.act(states[0],add_noise=False))
print(agent.act(states[1],add_noise=False))
print(agent.act(states[2],add_noise=False))

[ 0.04526401 -0.01214357 -0.05611927 -0.0407037 ]
[ 0.04178694  0.00143572 -0.06798373 -0.04478382]
[ 0.05150839 -0.01065964 -0.05387822 -0.03933354]


### 3. Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ",device)

using device:  cuda:0


In [11]:
data = np.load('state_scale.npz')
scale = data['scale']

env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)

action = agent.act(states[0]/scale,add_noise=False) 
action2 = agent.act(states[0]/scale,add_noise=False) 
actions = agent.act(states/scale,add_noise=False) 

print(action)
print(action2)
print(actions[0:2])

print(states[10]/scale)
all_states_scaled = states/scale
print(all_states_scaled[10])



[ 0.04854593 -0.00972608 -0.05750951 -0.04245726]
[ 0.04854593 -0.00972608 -0.05750951 -0.04245726]
[[ 0.04854593 -0.00972608 -0.05750951 -0.04245726]
 [ 0.04863288 -0.00939922 -0.05791854 -0.04218458]]
[  0.00000000e+00  -9.98329692e-01   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -9.98642193e-01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+00  -1.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   2.74895161e-01]
[  0.00000000e+00  -9.98329692e-01   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00  

In [12]:
# load weights after system crashed again :(
#load weights from last checkpoint
NN_architecture = None
Noise_params = None # default 0.0 0.15 0.2
rnd_seed = 42
#torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
agent = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, random_seed=rnd_seed, NN_architecture=NN_architecture)
agent.critic_local.load_state_dict(torch.load('Multi_checkpoint_critic.pth'))
agent.critic_target.load_state_dict(torch.load('Multi_checkpoint_critic.pth'))
agent.actor_local.load_state_dict(torch.load('Multi_checkpoint_actor.pth'))
agent.actor_target.load_state_dict(torch.load('Multi_checkpoint_actor.pth'))
#https://knowledge.udacity.com/questions/805381

Hyperparams 1000000 128 0.99 0.001 0.0001 0.001 0 new update 30 20
Hyperparams Noise 0.0 0.15 0.2


In [13]:
states = env_info.vector_observations  
rewards = env_info.rewards                         # get reward (for each agent)# get the current state (for each agent)
next_states = env_info.vector_observations         # get next state (for each agent)   
dones = env_info.local_done                        # see if episode finished
agent.step(states[0], action, rewards[0], next_states[0], dones[0])

In [ ]:
print('hey ho - lets go')
#NN_architecture = [64,64,32,16]  # actor_fc, critic_fc1, critic_fc2, critic_fc3
#NN_architecture = [64,64,32,16]  # actor_fc, critic_fc1, critic_fc2, critic_fc3
NN_architecture = None  # default 256,256,256,128

#Noise_params = [0.0,0.03,0.03]
Noise_params = None # default 0.0 0.15 0.2

rnd_seed = 42

agent = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, random_seed=rnd_seed, NN_architecture=NN_architecture, Noise_params=Noise_params, ReLU=True)

load_weights = True
if load_weights:
    print('weights loaded from last checkpoint')
    agent.critic_local.load_state_dict(torch.load('Multi_checkpoint_critic_2nd.pth'))
    agent.critic_target.load_state_dict(torch.load('Multi_checkpoint_critic_2nd.pth'))
    agent.actor_local.load_state_dict(torch.load('Multi_checkpoint_actor_2nd.pth'))
    agent.actor_target.load_state_dict(torch.load('Multi_checkpoint_actor_2nd.pth'))      
    '''
    # load 20 agent - every step update 250 steps checkpoint as start - CPU / own PC 
    # differnt PyTorch version - problems to load checkpoitn :( 
    agent.actor_local.load_state_dict(torch.load('checkpoint_actor_250.pth'))
    agent.actor_target.load_state_dict(torch.load('checkpoint_actor_250.pth'))
    agent.critic_local.load_state_dict(torch.load('checkpoint_critic_250.pth'))
    agent.critic_target.load_state_dict(torch.load('checkpoint_critic_250.pth'))        
    '''
else:
    print('Start training from scratch')
    

print("using device: ",device)
print('Network:',NN_architecture)
do_scaling = True
if do_scaling:
    print('load state scaling')
    data = np.load('state_scale.npz')
    #scale = data['scale_int']
    scale = data['scale']
else:
    scale = np.ones(33,)

add_noise2state = True
print('add_noise to state',add_noise2state)
if add_noise2state:
    print('Noise_params:',Noise_params)

def ddpg(n_episodes=2000, max_t=999):  # 2000 / 700
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):                        
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        #state = states[0]/scale
        states = states/scale
        agent.reset()  # noise reset....
        score = 0
        #############
        for t in range(max_t):
            actions = agent.act(states,add_noise=add_noise2state)      
            #print('t:',action)
            env_info = env.step(actions)[brain_name]           # send all action to Env
            next_states = env_info.vector_observations         # get next state (for each agent)   
            next_states = next_states/scale
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            #next_state, reward, done, _ = env.step(action)                                    
            ### -> agent.step(states[0]/scale, action, rewards[0], next_states[0]/scale, dones[0])
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done)
            states = next_states
            score += np.mean(rewards)
            if np.any(dones==True):
                print('SHOULD NEVER BE REACHED...')
                assert 1==0
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 50 == 0:
            torch.save(agent.actor_local.state_dict(), 'Multi_checkpoint_actor_30_20_local.pth')
            torch.save(agent.critic_local.state_dict(), 'Multi_checkpoint_critic_30_20_local.pth')
            torch.save(agent.actor_target.state_dict(), 'Multi_checkpoint_actor_30_20_target.pth')
            torch.save(agent.critic_target.state_dict(), 'Multi_checkpoint_critic_30_20_target.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
            # average score of +30 over 100 consecutive episodes
            if np.mean(scores_deque) >= 30.0:
                print('Assignment -DONE-')
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

hey ho - lets go
Hyperparams 1000000 128 0.99 0.001 0.0001 0.001 0 new update 30 20
Hyperparams Noise 0.0 0.15 0.2
weights loaded from last checkpoint
using device:  cuda:0
Network: None
load state scaling
add_noise to state True
Noise_params: None


/home/workspace/Multi_DDPG_agent.py:163: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)   # take advise from Attempt 3 / Udacity Course


Episode 35	Average Score: 5.47	Score: 5.47

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
action = agent.act(states[0]/scale) 
print(states[0]/scale)
print(states[0])
print(action)

In [ ]:
# some random agent - determine length of episodes...
done_mean = []
score_mean = []
states_mean = []
for runs in range(100):
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    jj = 0 
    while True:
        jj += 1 
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        states_mean.append(states[0])
        if np.any(dones):                                  # exit loop if episode finished
            #print(jj,'steps before done')
            done_mean.append(jj)
            break        
    #print(runs,'Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
    score_mean.append(np.mean(scores))

In [ ]:
print('<score>',np.mean(score_mean),'+/-',np.std(score_mean))
print('<done steps>',np.mean(done_mean),'+/-',np.std(done_mean))
print(states_mean[0].shape)
print(len(states_mean))
print(states_mean[10010-1])
states_np = np.array(states_mean)

s=0
print(np.argmax(states_np[:,s]))
print(states_mean[np.argmax(states_np[:,s])])
print(states_np.shape)

In [46]:
state = states_np[0]
dx = np.zeros(state.shape[0])
for s in range(state.shape[0]):
    if np.min(states_np[:][s]) > 0: 
        dx[s] = np.max(states_np[:,s])
    else:
        if np.abs(np.max(states_np[:,s])) > np.abs(np.min(states_np[:,s])):
            dx[s] = np.abs(np.max(states_np[:,s]))
        else:
            dx[s] = np.abs(np.min(states_np[:,s]))
    print('state ',s,'min/max scaling:',np.min(states_np[:,s]),np.max(states_np[:,s]),'\t\t\t->',np.min(states_np[:,s])/dx[s],np.max(states_np[:,s])/dx[s])
    #print('state ',s,':',dx[s])


state  0 min/max scaling: -4.020280838012695 4.010631561279297 			-> -1.0 0.997599850079586
state  1 min/max scaling: -4.006692409515381 1.8407402038574219 			-> -1.0 0.4594164002921462
state  2 min/max scaling: -3.9615864753723145 4.008120059967041 			-> -0.9883901719762583 1.0
state  3 min/max scaling: -0.1462584137916565 1.0 			-> -0.1462584137916565 1.0
state  4 min/max scaling: -0.6754260659217834 0.6729655265808105 			-> -1.0 0.9963570559901106
state  5 min/max scaling: -0.70906001329422 0.9998599886894226 			-> -0.7091593036177276 1.0
state  6 min/max scaling: -0.743294358253479 0.8522251844406128 			-> -0.8721807003877334 1.0
state  7 min/max scaling: -10.913050651550293 9.352399826049805 			-> -1.0 0.8569922494331331
state  8 min/max scaling: -2.058267831802368 1.7443766593933105 			-> -1.0 0.8474974113868399
state  9 min/max scaling: -3.3310513496398926 3.4418137073516846 			-> -0.9678186075338114 1.0
state  10 min/max scaling: -13.723333358764648 13.842732429504395 			-> -0.

C:\ProgramData\Anaconda3\envs\drlnd\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [82]:
scale = np.copy(dx)
scale[np.where(dx<1)]= 1
print(scale)

import math
scale_int = np.ones(scale.shape,dtype=int)
for jj in range(33):
  scale_int[jj] = math.ceil(scale[jj])
print(scale_int)

np.savez('state_scale.npz',scale=scale, scale_int=scale_int)

[ 4.02028084  4.00669241  4.00812006  1.          1.          1.
  1.         10.91305065  2.05826783  3.44181371 13.84273243  9.51571655
 13.86110687  9.92453194 10.01359653  9.82462692  1.          1.
  1.          1.         11.61246014  8.71936607  7.06147718 17.99956512
 19.19440842 16.99439049  8.          1.          8.          1.
  1.          1.          1.        ]
[ 5  5  5  1  1  1  1 11  3  4 14 10 14 10 11 10  1  1  1  1 12  9  8 18
 20 17  8  1  8  1  1  1  1]


In [85]:
data = np.load('state_scale.npz')
print(data['scale'])
print(data['scale_int'])

[ 4.02028084  4.00669241  4.00812006  1.          1.          1.
  1.         10.91305065  2.05826783  3.44181371 13.84273243  9.51571655
 13.86110687  9.92453194 10.01359653  9.82462692  1.          1.
  1.          1.         11.61246014  8.71936607  7.06147718 17.99956512
 19.19440842 16.99439049  8.          1.          8.          1.
  1.          1.          1.        ]
[ 5  5  5  1  1  1  1 11  3  4 14 10 14 10 11 10  1  1  1  1 12  9  8 18
 20 17  8  1  8  1  1  1  1]


### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [4]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
agent.reset()   
while True:
    action = agent.act(state)
    env.render()
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break
        
env.close()

### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  